In [38]:
import os
import pandas as pd
from tqdm import tqdm

In [28]:
db_path = '/app/nfs_clientshare/Datasets/Cosmenet_product_20240220'
new_db_path = '/app/nfs_clientshare/Datasets/cosmenet_scanproduct_clean'

In [35]:
def is_barcode(file):
    if set('arcodบาร์์ค้ด') & set(file.lower()):
        return True
    return False

In [30]:
def check_dir_exit(id, model):
    if not os.path.exists(f'{new_db_path}/{model}/{id}'):
        os.makedirs(f'{new_db_path}/{model}/{id}')

In [37]:
for root, dirs, files in tqdm(os.walk(db_path)):
    for file in files:
        product_id = os.path.basename(root)
        if is_barcode(file):
            check_dir_exit(product_id, 'barcode')
            os.rename(f'{root}/{file}', f'{new_db_path}/barcode/{product_id}/{file}')
        else:
            check_dir_exit(product_id, 'product')
            os.rename(f'{root}/{file}', f'{new_db_path}/product/{product_id}/{file}')

6726it [00:02, 2363.33it/s]


In [45]:
product_path = new_db_path + '/product'
ls_data = []
for root, dirs, files in tqdm(os.walk(product_path)):
    for file in files:
        id = os.path.basename(root)
        image_path = root+'/'+file
        ls_data.append([id, image_path])

6631it [00:03, 1929.54it/s]


In [54]:
ls_data = pd.DataFrame(ls_data, columns=['id', 'image_path'])
ls_data['id'] = ls_data['id'].astype(int)

In [46]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from IPython.display import display ,HTML
import requests, os
from PIL import Image
from io import BytesIO
from pyzbar.pyzbar import decode
from tqdm.auto import tqdm
from pathlib import Path

b_file_path = 'raw_data_production/b_file.csv'
b_iblock_element_path = 'raw_data_production/b_iblock_element.csv'
b_iblock_element_property_path = 'raw_data_production/b_iblock_element_property.csv'
b_iblock_property_path = 'raw_data_production/b_iblock_property.csv'
b_iblock_section_element_path = 'raw_data_production/b_iblock_section_element.csv'
b_iblock_section_path = 'raw_data_production/b_iblock_section.csv'
b_iblock_path = 'raw_data_production/b_iblock.csv'

b_iblock_element_property = pd.read_csv(b_iblock_element_property_path, usecols=['ID', 'IBLOCK_PROPERTY_ID', 'IBLOCK_ELEMENT_ID', 'VALUE'])
b_file = pd.read_csv(b_file_path, usecols=['ID', 'SUBDIR', 'FILE_NAME', 'ORIGINAL_NAME'])
b_iblock_element = pd.read_csv(b_iblock_element_path, usecols=['ID', 'IBLOCK_ID', 'IBLOCK_SECTION_ID', 'ACTIVE', 'NAME', 'PREVIEW_PICTURE', 'TIMESTAMP_X'])
b_iblock_property = pd.read_csv(b_iblock_property_path, usecols=['ID', 'IBLOCK_ID', 'NAME', 'ACTIVE'])
b_iblock_section_element = pd.read_csv(b_iblock_section_element_path, usecols=['IBLOCK_SECTION_ID', 'IBLOCK_ELEMENT_ID'])
b_iblock_section = pd.read_csv(b_iblock_section_path, usecols=['ID', 'IBLOCK_ID', 'IBLOCK_SECTION_ID', 'NAME', 'ACTIVE', 'DEPTH_LEVEL', 'TIMESTAMP_X'])
b_iblock = pd.read_csv(b_iblock_path, usecols=['ID', 'IBLOCK_TYPE_ID', 'NAME'])

url_404 = 'https://viterbischool.usc.edu/wp-content/uploads/2023/10/404.jpg'
oops_url = 'https://colorlib.com/wp/wp-content/uploads/sites/2/404-error-page-templates.jpg'
cosmenet_upload_url_prefix = 'https://www.cosmenet.in.th/upload/'
product_url_prefix = 'https://www.cosmenet.in.th/product/'

# Filter only brand and drop maker
brand_and_maker_iblock_index = b_iblock_section['IBLOCK_ID'] == 17
brand_depth_index = b_iblock_section['DEPTH_LEVEL'] == 2
brand_data = b_iblock_section[brand_and_maker_iblock_index & brand_depth_index]
brand_data = brand_data.drop(columns=['IBLOCK_ID', 'DEPTH_LEVEL'])
brand_data.rename(columns={'ID': 'BID', 'NAME': 'BRAND_NAME', 'IBLOCK_SECTION_ID':'MID'}, inplace=True)
brand_data = brand_data.astype({'MID': 'int32'})
# Check for duplicate brand name
brand_duplicate_unique = brand_data[brand_data['BRAND_NAME'].duplicated()].sort_values(by='BRAND_NAME')['BRAND_NAME'].unique()
brand_duplicate = brand_data[brand_data['BRAND_NAME'].isin(brand_duplicate_unique)]\
    .sort_values(['BRAND_NAME', 'BID'])[['BID', 'BRAND_NAME']]
# brand not active
brand_not_active_index = brand_data['ACTIVE'] == 'N'
brand_not_active_data = brand_data[brand_not_active_index]
brand_not_active_data = brand_not_active_data.drop(columns=['ACTIVE'])
# brand active
brand_active_index = brand_data['ACTIVE'] == 'Y'
brand_active_data = brand_data[brand_active_index]
brand_active_data = brand_active_data.drop(columns=['ACTIVE'])

brand_data.reset_index(drop=True, inplace=True)

# Filter only category and subcategory
cat_and_subcat_block_index = b_iblock_section['IBLOCK_ID'] == 6
cat_subcat_data = b_iblock_section[cat_and_subcat_block_index]
cat_subcat_data = cat_subcat_data.drop(columns=['IBLOCK_ID'])
# not active
cat_subcat_not_active_index = cat_subcat_data['ACTIVE'] == 'N'
cat_subcat_not_active_data = cat_subcat_data[cat_subcat_not_active_index]
cat_subcat_not_active_data = cat_subcat_not_active_data.drop(columns=['ACTIVE'])
# active
cat_subcat_active_index = cat_subcat_data['ACTIVE'] == 'Y'
cat_subcat_active_data = cat_subcat_data[cat_subcat_active_index]
cat_subcat_active_data = cat_subcat_active_data.drop(columns=['ACTIVE'])

# Filter only category and drop subcategory
category_data = cat_subcat_data[cat_subcat_data['DEPTH_LEVEL'] == 1]
category_data = category_data.drop(columns=['DEPTH_LEVEL', 'IBLOCK_SECTION_ID'])
category_data.rename(columns={'ID': 'CID', 'NAME': 'CATEGORY_NAME'}, inplace=True)
category_data.reset_index(drop=True, inplace=True)

# Merge category and subcategory
subcategory_data = pd.merge(cat_subcat_data, cat_subcat_data, left_on='IBLOCK_SECTION_ID', right_on='ID')
midcategory_index = subcategory_data['DEPTH_LEVEL_x'] == 2
subcategory_data = subcategory_data[~midcategory_index]
subcategory_data.drop(columns=['DEPTH_LEVEL_x', 'DEPTH_LEVEL_y', 'ID_y', 'NAME_y', 'IBLOCK_SECTION_ID_x', 'TIMESTAMP_X_y', 'ACTIVE_y'], inplace=True)
subcategory_data.rename(columns={'ID_x': 'SCID', 'NAME_x': 'SUBCATEGORY_NAME', 'IBLOCK_SECTION_ID_y': 'CID', 
                                 'TIMESTAMP_X_x':'TIMESTAMP_X', 'ACTIVE_x':'ACTIVE'}, inplace=True)
subcategory_data = subcategory_data.astype({'CID': 'int32'})
subcategory_data.reset_index(drop=True, inplace=True)

# Filter only product
product_block_index = b_iblock_element['IBLOCK_ID'] == 17
product_data = b_iblock_element[product_block_index]
product_data = product_data.drop(columns=['IBLOCK_ID'])
product_data.rename(columns={'ID': 'EID', 'IBLOCK_SECTION_ID': 'BID', 'NAME': 'PRODUCT_NAME'}, inplace=True)
# Check brand not provided
product_not_BID = product_data[product_data['BID'].isna()]
product_data = product_data.astype({'BID': 'int32'})
# create product url
product_data['PRODUCT_URL'] = product_data['EID'].apply(lambda x: product_url_prefix + str(x))
# merge product with category and subcategory
subcat_link_data = pd.merge(b_iblock_section_element, subcategory_data.drop(columns=['ACTIVE']), how='right', left_on='IBLOCK_SECTION_ID', right_on='SCID')
subcat_link_data.drop(columns=['IBLOCK_SECTION_ID', 'SUBCATEGORY_NAME', 'CID', 'TIMESTAMP_X'], inplace=True)
product_data_linked= pd.merge(product_data, subcat_link_data, how='left', left_on='EID', right_on='IBLOCK_ELEMENT_ID')
product_data_linked.drop(columns=['IBLOCK_ELEMENT_ID'], inplace=True)
# create product preview image
product_data_linked = pd.merge(product_data_linked, b_file, left_on='PREVIEW_PICTURE', right_on='ID', how='left')

product_data_linked['PREVIEW_PICTURE'] = product_data_linked.apply(lambda x: cosmenet_upload_url_prefix + x['SUBDIR'] 
                                                                   if not pd.isna(x['PREVIEW_PICTURE']) 
                                                                   else None, axis=1)
def check_url(x):
    if not pd.isna(x['FILE_NAME']):
        return x['PREVIEW_PICTURE'] + '/' + x['FILE_NAME']
    else:
        return None
product_data_linked['PREVIEW_PICTURE'] = product_data_linked.apply(lambda x: check_url(x), axis=1)
product_data_linked.drop(columns=['ID', 'SUBDIR', 'ORIGINAL_NAME'], inplace=True)
# active
product_active_index = product_data_linked['ACTIVE'] == 'Y'
product_active_data = product_data_linked[product_active_index]
product_active_data = product_active_data.drop(columns=['ACTIVE'])
product_active_data = product_active_data.astype({'SCID': 'int32'})
# not active
product_not_active_index = product_data_linked['ACTIVE'] == 'N'
product_not_active_data = product_data_linked[product_not_active_index]
product_not_active_data = product_not_active_data.drop(columns=['ACTIVE'])

product_data_linked.reset_index(drop=True, inplace=True)
product_data_linked.head(1)

,EID,TIMESTAMP_X,BID,ACTIVE,PRODUCT_NAME,PREVIEW_PICTURE,PRODUCT_URL,SCID,FILE_NAME
0,102,2022-06-28 20:09:51,1497,Y,Beneficial Make Off Perfect Eye & Lip Makeup R...,https://www.cosmenet.in.th/upload/iblock/b09/B...,https://www.cosmenet.in.th/product/102,73.0,Beneficial-Make-Off-Perfect-Eye-Lip-Makeup-Rem...


In [57]:
ls_data[~(ls_data['id'].isin(product_data_linked['EID']))]

,id,image_path


In [59]:
from PIL import Image

In [60]:
for index, row in tqdm(ls_data.iterrows()):
    try:
        img = Image.open(row['image_path'])
        img.verify()
    except:
        print(row['image_path'])

0it [00:00, ?it/s]

/app/nfs_clientshare/Datasets/cosmenet_scanproduct_clean/product/36977/36977_10.jpg
/app/nfs_clientshare/Datasets/cosmenet_scanproduct_clean/product/37041/37041_1.jpg
/app/nfs_clientshare/Datasets/cosmenet_scanproduct_clean/product/37041/37041_10.jpg
